<a href="https://colab.research.google.com/github/devaru-ai/cuda-inference/blob/main/SIMT-Kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.1 Prove that variables are expensive**

## Skinny Kernel

1. Thread with small no of registers
2. High Occupancy

## Fat Kernel

1. Thread declares 20 local variables
2. Crashes at ~5-10%

# **Try 1 : We got a silent crash!**

We use volatile keyword to force the compiler to be dumb.

Even if its not used, we tell it to store the variable in a register.

We create a dependency chain using the for loop to keep all those variables alive in the register. Eg: v1 += v2 ie, to update v1, it needs v2.

# **Debug:**

## **The 0.0% Occupancy.**

Func Call: cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, ...);

- I'm running on A100.

- By default nvcc often compiles for old architecture.

- So, A100 driver looked at code and was like "Bruhh.. this code is compiled for an old GPU, I can't calculate for A100 using this old blueprint."

- The func failed.

- Bcz, the func failed, it never wrote a val into numBlocks.

- numBlocks remained at its default 0.

- The math: 0 * 256 * 100/MaxThreads = 0%.

**So, next we are gonna add the flag --arch=sm_80.**

In [3]:
%%writefile register_test.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- KERNEL 1: The "Lean" Kernel (Small Luggage) ---
// Uses minimal variables. Should fit many threads.
__global__ void lean_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;

    // Just one variable. The compiler loves this.
    float val = tid * 1.0f;
    val += 10.0f;
    out[tid] = val;
}

// --- KERNEL 2: The "Fat" Kernel (Big Luggage) ---
// Uses tons of variables. The SM will run out of space.
__global__ void fat_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;

    // "volatile" tells the compiler: "Do NOT optimize these away. Keep them in registers."
    // We are forcing each thread to hold onto 16 different numbers at once.
    volatile float v0 = tid * 1.0f;
    volatile float v1 = tid * 2.0f;
    volatile float v2 = tid * 3.0f;
    volatile float v3 = tid * 4.0f;
    volatile float v4 = tid * 5.0f;
    volatile float v5 = tid * 6.0f;
    volatile float v6 = tid * 7.0f;
    volatile float v7 = tid * 8.0f;
    volatile float v8 = tid * 9.0f;
    volatile float v9 = tid * 10.0f;
    volatile float v10 = tid * 11.0f;
    volatile float v11 = tid * 12.0f;
    volatile float v12 = tid * 13.0f;
    volatile float v13 = tid * 14.0f;
    volatile float v14 = tid * 15.0f;
    volatile float v15 = tid * 16.0f;

    // Fake math loop to keep variables alive
    for(int i=0; i<100; i++) {
        v0 += v1; v1 += v2; v2 += v3; v3 += v4;
        v4 += v5; v5 += v6; v6 += v7; v7 += v8;
        v8 += v9; v9 += v10; v10 += v11; v11 += v12;
        v12 += v13; v13 += v14; v14 += v15; v15 += v0;
    }

    out[tid] = v0 + v1 + v2 + v3;
}

int main() {
    int dev_id = 0;
    cudaSetDevice(dev_id);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, dev_id);

    printf("GPU Model: %s\n", prop.name);
    printf("Total Register Capacity per SM: %d\n", prop.regsPerMultiprocessor);
    printf("------------------------------------------------\n");

    int numBlocks;
    int blockSize = 256; // Standard block size

    // --- MEASURE LEAN KERNEL ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, lean_kernel, blockSize, 0);

    float occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[LEAN KERNEL] Max Blocks per SM: %d  |  Occupancy: %.1f%%\n", numBlocks, occupancy);

    // --- MEASURE FAT KERNEL ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, fat_kernel, blockSize, 0);

    occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[FAT KERNEL]  Max Blocks per SM: %d  |  Occupancy: %.1f%%\n", numBlocks, occupancy);

    printf("------------------------------------------------\n");
    return 0;
}

Overwriting register_test.cu


In [4]:
!nvcc register_test.cu -o register_test
!./register_test

GPU Model: NVIDIA A100-SXM4-40GB
Total Register Capacity per SM: 65536
------------------------------------------------
[LEAN KERNEL] Max Blocks per SM: 0  |  Occupancy: 0.0%
[FAT KERNEL]  Max Blocks per SM: 0  |  Occupancy: 0.0%
------------------------------------------------


# **Try 2: A100 is quite the beast!**

Basically, it saw the Fat Kernel, and was like, Is that all you got?

# **The Math**

## **The Hardware Limit (A100 Stats)**

1. Total Reg per SM = 65,536
2. Max Threads allowed per SM = 2,048
3. To hit 100% occupancy: each thread must use fewer than 65,536/2,048 = 32 registers.

I declared 16 variables, that's 16-20 registers (16 for var + a few for loop indices)

Is 20 < 32? Hmm, That's skinny for A100.

So, 20 * 2048 = 40,960 registers.

We have 65,536 registers available.

Next, we are gonna force the code to use 72 registers.

# **The Big Learning: Register Pressure is Relative.**

On older GPU like T4 or V100, the 16-var code might have dropped occupancy to 75% or 50%.

A100 is a very forgiving chip.

In [5]:
%%writefile register_test.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- KERNEL 1: Lean ---
__global__ void lean_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    float val = tid * 1.0f;
    val += 10.0f;
    out[tid] = val;
}

// --- KERNEL 2: Fat ---
__global__ void fat_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;

    // Force high register usage
    volatile float v0 = tid * 1.0f; volatile float v1 = tid * 2.0f;
    volatile float v2 = tid * 3.0f; volatile float v3 = tid * 4.0f;
    volatile float v4 = tid * 5.0f; volatile float v5 = tid * 6.0f;
    volatile float v6 = tid * 7.0f; volatile float v7 = tid * 8.0f;
    volatile float v8 = tid * 9.0f; volatile float v9 = tid * 10.0f;
    volatile float v10 = tid * 11.0f; volatile float v11 = tid * 12.0f;
    volatile float v12 = tid * 13.0f; volatile float v13 = tid * 14.0f;
    volatile float v14 = tid * 15.0f; volatile float v15 = tid * 16.0f;

    // Fake math loop
    for(int i=0; i<100; i++) {
        v0 += v1; v1 += v2; v2 += v3; v3 += v4;
        v4 += v5; v5 += v6; v6 += v7; v7 += v8;
        v8 += v9; v9 += v10; v10 += v11; v11 += v12;
        v12 += v13; v13 += v14; v14 += v15; v15 += v0;
    }
    out[tid] = v0 + v1 + v2 + v3;
}

int main() {
    int dev_id = 0;
    cudaSetDevice(dev_id);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, dev_id);

    printf("GPU Model: %s\n", prop.name);

    int numBlocks;
    int blockSize = 256;
    cudaError_t err;

    // --- LEAN KERNEL ---
    err = cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, lean_kernel, blockSize, 0);
    if (err != cudaSuccess) {
        printf("FAILED (Lean): %s\n", cudaGetErrorString(err));
        return -1;
    }

    float occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[LEAN KERNEL] Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    // --- FAT KERNEL ---
    err = cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, fat_kernel, blockSize, 0);
    if (err != cudaSuccess) {
        printf("FAILED (Fat): %s\n", cudaGetErrorString(err));
        return -1;
    }

    occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[FAT KERNEL]  Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    return 0;
}

Overwriting register_test.cu


In [6]:
!nvcc -arch=sm_80 register_test.cu -o register_test
!./register_test

GPU Model: NVIDIA A100-SXM4-40GB
[LEAN KERNEL] Blocks: 8 | Occupancy: 100.0%
[FAT KERNEL]  Blocks: 8 | Occupancy: 100.0%


# **Try 3: I tried to trick the Compile into using 100 registers**

## It analysed the logic and realised it could do the exact same math using only 2 registers.

# **The Bluff**

float data[100];

- I thought it would reserve space for 100 sep numbers.


Fatal Flaw:
data[i+1] += data[i] * 0.5f;

- Calculate data[0]. Put it in R1
- Next, data[1] = data[1] + (data[0] * 0.5)
- Read R1, do the math, put it in R2.
- Looks ahead, Does anybody need data[0] (R1) ever again? No? Nobody.. Marks R1 as FREE.

So, instead of 100 registers like I thought it would, it just used around ~10 (for math + indices). So, Occupancy: 100%


In [7]:
%%writefile register_test.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- KERNEL 1: Lean (Control Group) ---
__global__ void lean_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    float val = tid * 1.0f;
    val += 10.0f;
    out[tid] = val;
}

// --- KERNEL 2: Super Fat (Experimental Group) ---
// We use a local array of 100 floats.
// If the compiler puts this in registers, it uses ~100 registers per thread.
// 100 regs * 2048 threads = 200,000 regs needed.
// The SM only has 65,536.
// RESULT: The SM MUST reduce the number of active threads (Occupancy crash).
__global__ void fat_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;

    // Declare a massive amount of local state
    float data[100];

    // Initialize (prevent optimization)
    #pragma unroll
    for(int i=0; i<100; i++) {
        data[i] = tid * 0.001f + i;
    }

    // Heavy computation to keep registers "alive"
    #pragma unroll
    for(int i=0; i<99; i++) {
        data[i+1] += data[i] * 0.5f;
    }

    out[tid] = data[99];
}

int main() {
    int dev_id = 0;
    cudaSetDevice(dev_id);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, dev_id);

    printf("GPU: %s\n", prop.name);

    int numBlocks;
    int blockSize = 256;

    // --- LEAN ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, lean_kernel, blockSize, 0);
    float occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[LEAN KERNEL] Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    // --- FAT ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, fat_kernel, blockSize, 0);
    occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[FAT KERNEL]  Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    return 0;
}

Overwriting register_test.cu


In [8]:
!nvcc -arch=sm_80 --ptxas-options=-v register_test.cu -o register_test
!./register_test

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z10fat_kernelPfi' for 'sm_80'
ptxas info    : Function properties for _Z10fat_kernelPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 364 bytes cmem[0]
ptxas info    : Compiling entry function '_Z11lean_kernelPfi' for 'sm_80'
ptxas info    : Function properties for _Z11lean_kernelPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 8 registers, 364 bytes cmem[0]
GPU: NVIDIA A100-SXM4-40GB
[LEAN KERNEL] Blocks: 8 | Occupancy: 100.0%
[FAT KERNEL]  Blocks: 8 | Occupancy: 100.0%


# **Try 4: Victoryyy!**

1. Declare 128 vars
2. #pragma unroll -> Instead of using a loop counter, we make the compiler copy paste the lines 128 times, creating 128 independent assignments.
3. Forces compiler to treat val[0] through val[127] as sep vars(registers)

- The Logic:

1. When i = 0: We update vals[0] using vals[64].

2. When i = 64: We update vals[64] using vals[0].


# **The Math**

72 reg per thread

156 threads * 72 regs = 18,432 regs per block

Total SM capacity = 65, 536 registers

65,536 / 18, 432 = 3.55

GPU rounds down to 3 blocks

3 blocks * 256 threads = 768 threads

768 / 2048 (Max) = 37.5%

# **Conclusion**

By forcing each thread to carry "72 registers", the SM could only fit a few threads, leaving 60%+ empty.

In [9]:
%%writefile register_test.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- KERNEL 1: Lean ---
__global__ void lean_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    float val = tid * 1.0f;
    val += 10.0f;
    out[tid] = val;
}

// --- KERNEL 2: Fat ---
// Goal: Burn > 64 registers per thread to crash occupancy below 50%
__global__ void fat_kernel(float *out, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;

    // 1. Declare 128 variables (Massive state)
    float vals[128];

    // 2. Initialize them (Prevent dead code elimination)
    #pragma unroll
    for (int i = 0; i < 128; i++) {
        vals[i] = tid * 0.0001f + i;
    }

    // 3. The "Liveness" Trap
    // We update vals[i] using vals[i + 64].
    // This forces the compiler to keep 'vals[0]' alive until we reach 'vals[64]'.
    // It CANNOT reuse the register for vals[0] yet.
    #pragma unroll
    for (int k = 0; k < 5; k++) { // Repeat to add complexity
        #pragma unroll
        for (int i = 0; i < 128; i++) {
            int neighbor = (i + 64) % 128; // Look 64 steps away
            vals[i] += vals[neighbor] * 0.001f;
        }
    }

    // 4. Sum them up (Output dependency)
    float sum = 0.0f;
    #pragma unroll
    for (int i = 0; i < 128; i++) {
        sum += vals[i];
    }

    out[tid] = sum;
}

int main() {
    int dev_id = 0;
    cudaSetDevice(dev_id);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, dev_id);

    printf("GPU: %s\n", prop.name);

    int numBlocks;
    int blockSize = 256;

    // --- LEAN ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, lean_kernel, blockSize, 0);
    float occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[LEAN KERNEL] Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    // --- FAT ---
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, fat_kernel, blockSize, 0);
    occupancy = (numBlocks * blockSize * 100.0f) / prop.maxThreadsPerMultiProcessor;
    printf("[FAT KERNEL]  Blocks: %d | Occupancy: %.1f%%\n", numBlocks, occupancy);

    return 0;
}

Overwriting register_test.cu


In [10]:
!nvcc -arch=sm_80 --ptxas-options=-v register_test.cu -o register_test
!./register_test

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z10fat_kernelPfi' for 'sm_80'
ptxas info    : Function properties for _Z10fat_kernelPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 72 registers, 364 bytes cmem[0]
ptxas info    : Compiling entry function '_Z11lean_kernelPfi' for 'sm_80'
ptxas info    : Function properties for _Z11lean_kernelPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 8 registers, 364 bytes cmem[0]
GPU: NVIDIA A100-SXM4-40GB
[LEAN KERNEL] Blocks: 8 | Occupancy: 100.0%
[FAT KERNEL]  Blocks: 3 | Occupancy: 37.5%


# **1.2 Make Half Go Left and Half Go Right**

# **Warp Divergence**

# 1. Coherent Warp

We have 32 threads in a warp.
- if (tid < 32)
- It's gonna be True for them all.

The Warp executes the True block once.

**Time Taken: 1 Unit.**

# 2. Divergent Warp
- if (tid % 2 == 0)
- For even threads, its True
- For odd ones, it's False

The Warp has to serialize in this case.

**Step A = True Path:**
- It masks all odd ones
- Even threads works while odd ones sit idle.
- Time Taken: 1 Unit.

**Step B = False Path:**
- Hardware flips the mask, Now evens are masked.
- Even threads are idle, Odd ones work now.
- Time Taken: 1 Unit.

**Total Time: Step A (1 Unit) + Step B (1 Unit) = 2 Units.**


**You paid for 100% of the chip, but you only used 50% of it at any given moment.**

## Wasting Moneyyyy!


# I'm gonna tell something cool now, So Listen Close!

# It's PTX

Parallel Thread Execution.

- It's like the virtual assembly language for NVIDIA GPUs.

- So you have the C++/CUDA code - That's high level human logic

- Then you have PTX, essentially what the compiler (nvcc) produces. It's low level assembly lang, but it's generic. It doesn't know if you are running on GTX 1080 or an A100. It just describes the instructions.

- SASS (Haha, who named it?): It's Streaming Assembly. The absolute bottom layer. The GPU Driver produces binary that the silicon executes.

# PTX is a lie and SASS is the harsh reality.

- PTX pretends like you have infinite registers like %r9999 - valid!

- SASS knows the truth. It's like I got your map to infinite variables to fit into **256** physical slots R0 through R255.

But SASS is smart. And nice too! And it makes things possible, not complain!

# **Register Reallocation**
## One of the difficult but cool math problems
## It's not rocket science, It either does **Register reuse** after one operation or it does **Register Spilling**

# Register spilling is disastrous for performance.

- Say the code has 300 vars in one line of SASS code.
- It kicks out 45 back to global memory.
- Y'all know HBM/DRAM is slowwww.

- Register Access: ~1 Clock Cycle (On-chip, next to the ALU).

- Local Memory Access: ~200–600 Clock Cycles (Off-chip, traveling to VRAM and back).

- The Stall: The CUDA Core (ALU) sits idle for those 600 cycles, doing absolutely nothing, waiting for the data to arrive. This is why performance collapses.



Okay, I got distracted by PTX, I just wanted to talk abt __nanosleep(ns):

- It just tells Warp Scheduler to not issue any more instructions for this warp for X clock cycles

- So the warp stays active but it just goes to stall stanvte.

- nvcc is an aggressive optimizer, so when it sees my math loop it could be like you calculated a but you never used it later. This loop does nothing useful. **DELETE**. It erases the loop and it takes 0 ns.

- But if I insert nanosleep, it can't delete it cuz that would change the program behavior.

# **Try 1 : I got "Optimized, Compiler Wins"**

- The first kernel we launch in any program pays the Driver Tax. GPU had to wake up, load context and initialise the clock. So that took extra 0.38 ms. So, thats why coherent kernel took 1.38 ms.

- The Divergent kernel took 1.019 ms. That's like 1 trip. Not 2.

- The compiler looked at my code and was like Wait, if you are even you sleep, and if you are odd you sleep. I'm deleting if/else logic and just running sleep (1ms) for everyone.

- So, I wanted to split but it taped it back together.


## **Next try I'm gonna add a Warmup kernel and make the branches diff So compiler cannot merge them.**



In [3]:
%%writefile warp_divergence.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- HELPER: The Time Anchor ---
// This acts as a "heavy math operation" that takes exactly 1ms.
// The compiler cannot delete this because it has a side effect (time).
__device__ void burn_time() {
    __nanosleep(1000000); // 1,000,000 nanoseconds = 1 millisecond
}

// --- KERNEL A: Coherent (The Unanimous Vote) ---
// Condition: tid < 32
// Result: In a warp of 32 threads, ALL threads evaluate this to TRUE.
// Execution: The warp runs the TRUE block once.
// Cost: 1 Trip.
__global__ void coherent_kernel() {
    int tid = threadIdx.x;

    if (tid < 32) {
        burn_time();
    } else {
        burn_time(); // Dead code (never reached by this warp)
    }
}

// --- KERNEL B: Divergent (The Split Vote) ---
// Condition: tid % 2 == 0
// Result: 16 threads are Even (True), 16 threads are Odd (False).
// Execution:
//    1. Hardware masks off Odds. Runs Evens (1ms).
//    2. Hardware masks off Evens. Runs Odds (1ms).
// Cost: 2 Trips.
__global__ void divergent_kernel() {
    int tid = threadIdx.x;

    if (tid % 2 == 0) {
        burn_time();
    } else {
        burn_time();
    }
}

int main() {
    // 1. Setup Timers (CUDA Events are precise hardware timestamps)
    float time_coherent, time_divergent;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    printf("--- WARP DIVERGENCE BENCHMARK ---\n");
    printf("Task: Force the GPU to sleep for 1ms inside an if/else.\n");
    printf("Hypothesis: Divergent kernel should take 2x longer.\n");
    printf("-------------------------------------\n");

    // 2. Run Coherent Kernel (1 Block, 32 Threads = 1 Warp)
    cudaEventRecord(start);
    coherent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_coherent, start, stop);

    printf("Coherent Time:  %.3f ms\n", time_coherent);

    // 3. Run Divergent Kernel (1 Block, 32 Threads = 1 Warp)
    cudaEventRecord(start);
    divergent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_divergent, start, stop);

    printf("Divergent Time: %.3f ms\n", time_divergent);

    // 4. Validate
    float ratio = time_divergent / time_coherent;
    printf("-------------------------------------\n");
    printf("Slowdown Factor: %.2fx\n", ratio);

    if (ratio > 1.9) {
        printf("CONCLUSION: SUCCESS. The hardware serialized the execution paths.\n");
    } else {
        printf("CONCLUSION: FAILED. Check compiler settings.\n");
    }

    return 0;
}

Overwriting warp_divergence.cu


In [4]:
!nvcc -arch=sm_80 warp_divergence.cu -o warp_divergence
!./warp_divergence

--- WARP DIVERGENCE BENCHMARK ---
Task: Force the GPU to sleep for 1ms inside an if/else.
Hypothesis: Divergent kernel should take 2x longer.
-------------------------------------
Coherent Time:  1.382 ms
Divergent Time: 1.019 ms
-------------------------------------
Slowdown Factor: 0.74x
CONCLUSION: FAILED. Check compiler settings.


# **Try 2: A100 is mocking me**

- A100 has Independent Thread Scheduling.

- Old GPUs like Pascal, the warp was locked. If threads divereged, the hardware physically couldn't manage the others

- A100 maintains a separate Program Counter for every thread. While it should have serialized it allowed some overlap even inside a single warp.

## **Now I'm gonna stop asking nicely with nanosleep to scheduler, and I'm gonna force the threads to calculate math in a while loop using GPU's raw clock cycles (clock64()).**

The Math units ALU cannot be shared. If 16 threads are using the ALU the other 16 must wait. There's no magic scheduler that can create extra ALU's out of thin air.



In [5]:
%%writefile warp_divergence_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Helper: Freezes the core for 'ns' nanoseconds
__device__ void burn_time(long ns) {
    __nanosleep(ns);
}

// --- WARMUP KERNEL ---
// Wakes up the GPU so the first real test doesn't pay the penalty.
__global__ void warmup() {
    int tid = threadIdx.x;
    if (tid == 0) burn_time(10000);
}

// --- KERNEL A: Coherent ---
__global__ void coherent_kernel() {
    int tid = threadIdx.x;
    if (tid < 32) {
        burn_time(1000000); // 1ms
    } else {
        burn_time(1000000); // Dead code
    }
}

// --- KERNEL B: Divergent ---
__global__ void divergent_kernel() {
    int tid = threadIdx.x;

    if (tid % 2 == 0) {
        burn_time(1000000); // Evens: 1,000,000 ns
    } else {
        // TRICK: Change the value by 100ns.
        // The compiler sees different arguments and CANNOT merge the branches.
        // It is forced to generate two separate paths.
        burn_time(1000100); // Odds:  1,000,100 ns
    }
}

int main() {
    float time_coherent, time_divergent;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    printf("--- WARP DIVERGENCE BENCHMARK V2 ---\n");

    // 1. WARMUP (Crucial Step)
    warmup<<<1, 32>>>();
    cudaDeviceSynchronize(); // Wait for warmup to finish

    // 2. Measure Coherent
    cudaEventRecord(start);
    coherent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_coherent, start, stop);
    printf("Coherent Time:  %.3f ms\n", time_coherent);

    // 3. Measure Divergent
    cudaEventRecord(start);
    divergent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_divergent, start, stop);
    printf("Divergent Time: %.3f ms\n", time_divergent);

    // 4. Analysis
    float slowdown = time_divergent / time_coherent;
    printf("Slowdown Factor: %.2fx\n", slowdown);

    if (slowdown > 1.9) {
        printf("CONCLUSION: SUCCESS. Serialized (2 Trips).\n");
    } else {
        printf("CONCLUSION: FAILED. Still optimized.\n");
    }

    return 0;
}

Writing warp_divergence_v2.cu


In [6]:
!nvcc -arch=sm_80 warp_divergence_v2.cu -o warp_divergence_v2
!./warp_divergence_v2

--- WARP DIVERGENCE BENCHMARK V2 ---
Coherent Time:  0.762 ms
Divergent Time: 1.016 ms
Slowdown Factor: 1.33x
CONCLUSION: FAILED. Still optimized.


# **Try 3: A100, Take That!**

- clock64() reads the actual hardware tick. By forcing a while loop on it, we are jamming the execution pipeline. The "Odd" threads cannot use the pipeline until the "Even" threads release it.

In [7]:
%%writefile warp_divergence_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- THE SPIN LOCK ---
// Instead of sleeping, we BURN cycles.
// We continuously check the clock. This occupies the ALU 100%.
// It prevents the scheduler from doing anything else.
__device__ void burn_cycles(long long cycles) {
    long long start = clock64();
    while (clock64() - start < cycles) {
        // Spin!
    }
}

// --- KERNEL A: Coherent ---
// Everyone burns 10,000,000 cycles (~7-8ms on A100)
__global__ void coherent_kernel() {
    int tid = threadIdx.x;
    if (tid < 32) {
        burn_cycles(10000000);
    }
}

// --- KERNEL B: Divergent ---
// Evens burn 10M. Odds burn 10M.
// Since they share the SAME ALU execution unit, they MUST run sequentially.
__global__ void divergent_kernel() {
    int tid = threadIdx.x;

    if (tid % 2 == 0) {
        burn_cycles(10000000);
    } else {
        // We use a separate function call/block to ensure no merging
        burn_cycles(10000000);
    }
}

int main() {
    float time_coherent, time_divergent;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    printf("--- WARP DIVERGENCE: ALU SPIN TEST ---\n");
    printf("Task: Burn 10,000,000 clock cycles.\n");

    // 1. WARMUP
    coherent_kernel<<<1, 32>>>();
    cudaDeviceSynchronize();

    // 2. Measure Coherent
    cudaEventRecord(start);
    coherent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_coherent, start, stop);
    printf("Coherent Time:  %.3f ms\n", time_coherent);

    // 3. Measure Divergent
    cudaEventRecord(start);
    divergent_kernel<<<1, 32>>>();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_divergent, start, stop);
    printf("Divergent Time: %.3f ms\n", time_divergent);

    // 4. Analysis
    float ratio = time_divergent / time_coherent;
    printf("Slowdown Factor: %.2fx\n", ratio);

    if (ratio > 1.95) {
        printf("CONCLUSION: SUCCESS. The ALUs were serialized.\n");
    } else {
        printf("CONCLUSION: WTF.\n");
    }

    return 0;
}

Writing warp_divergence_v3.cu


In [8]:
!nvcc -arch=sm_80 warp_divergence_v3.cu -o warp_divergence_v3
!./warp_divergence_v3

--- WARP DIVERGENCE: ALU SPIN TEST ---
Task: Burn 10,000,000 clock cycles.
Coherent Time:  9.144 ms
Divergent Time: 18.298 ms
Slowdown Factor: 2.00x
CONCLUSION: SUCCESS. The ALUs were serialized.


# **Stride**:

In this code:

tid * 1: Every thread takes the very next number. (Walking).

tid * 32: Every thread jumps 32 spots away from its neighbor. (Long Jumping).

**Why it kills performance:** The Memory Controller hates Long Jumps. It has to calculate a new address for every single thread instead of just saying "Give me the whole block."



In [11]:
%%writefile coalescing_stride.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void stride_kernel(float *data, int stride, int N) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    long long idx = (long long)tid * stride;

    if (idx < N) {
        data[idx] += 1.0f;
    }
}

int main() {
    int N = 128 * 1024 * 1024; // 128 Million Floats
    size_t bytes = N * sizeof(float);
    float *d_data;
    cudaMalloc(&d_data, bytes);

    int strides[] = {1, 32}; // Comparing Best vs Worst

    printf("--- STRIDE BENCHMARK ---\n");

    for (int i = 0; i < 2; i++) {
        int s = strides[i];
        int elements_touched = N / s;
        int numBlocks = (elements_touched + 256 - 1) / 256;

        cudaEvent_t start, stop;
        cudaEventCreate(&start); cudaEventCreate(&stop);

        cudaEventRecord(start);
        stride_kernel<<<numBlocks, 256>>>(d_data, s, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);

        // Calculate Bandwidth
        double total_bytes = (double)elements_touched * 4 * 2;
        double gb_per_sec = (total_bytes / (milliseconds / 1000.0)) / 1e9;

        printf("Stride %2d:  %6.2f GB/s\n", s, gb_per_sec);
    }
    cudaFree(d_data);
    return 0;
}

Overwriting coalescing_stride.cu


In [12]:
!nvcc -arch=sm_80 coalescing_stride.cu -o coalescing_test
!./coalescing_test

--- STRIDE BENCHMARK ---
Stride  1:  1167.68 GB/s
Stride 32:   80.12 GB/s


# Project 1.3: The Single-Thread Juggler.

- Usually, GPUs hide latency by swapping threads (Thread Level Parallelism). If Thread 1 is waiting for memory, the GPU runs Thread 2. But what if you only have 1 thread?

- You have to use ILP (Instruction Level Parallelism). This is like a juggler.

- 1. Serial (Bad): Throw Ball 1. Watch it fly. Catch it. Throw Ball 2.

- 2. ILP (Good): Throw Ball 1. Throw Ball 2. Throw Ball 3. Catch Ball 1. Catch Ball 2...

- We are going to force 1 Warp to process a huge array.

- Kernel A: Loads one value, squares it, stores it. (Stop-and-Go).

- Kernel B: Loads 4 values at once, squares them, stores them. (Pipelined).

## **We expected Kernel B to be 2x to 4x faster.**

## **But here's an 8.47x speedup.**

- That happened due to Vectorization.
The Compiler saw we asked for x, y, z, w (4 floats) right next to each other. Instead of issuing 4 separate "Load Float" instructions, it fused them into One Giant "Load Float4" Instruction.

- Serial Kernel: 4 Instructions. 4 Address Calculations. 4 Latency penalties.

- ILP Kernel: 1 Instruction. 1 Address Calculation. 1 Latency penalty.

This didn't just hide latency; it slashed the instruction count by 75%.

i += stride

- stride is the Total Number of Threads in the grid (blockDim.x * gridDim.x)

- After a thread finishes its first job, it "jumps" forward by the total number of threads to find its next job.

In [13]:
%%writefile ilp_unrolling.cu
#include <stdio.h>
#include <cuda_runtime.h>

// --- KERNEL A: Serial (The One-Ball Juggler) ---
// Load -> Stall -> Math -> Store -> Stall -> Repeat
__global__ void serial_kernel(float *data, int N) {
    int tid = threadIdx.x;
    int stride = blockDim.x;

    for (int i = tid; i < N; i += stride) {
        float x = data[i];      // Load (Latency: 400 cycles)
        x = x * x;              // Math (Dependent on Load)
        data[i] = x;            // Store
    }
}

// --- KERNEL B: ILP Unrolled (The Four-Ball Juggler) ---
// Load 4 items. The GPU issues all 4 loads before the first one comes back.
// While waiting for 'x', it issues the request for 'y'.
__global__ void ilp_kernel(float *data, int N) {
    int tid = threadIdx.x;
    int stride = blockDim.x;

    // Process 4 elements per loop iteration
    for (int i = tid; i < N; i += stride * 4) {
        // 1. Issue 4 Loads immediately (Pipelining)
        float x = data[i];
        float y = data[i + stride];
        float z = data[i + stride * 2];
        float w = data[i + stride * 3];

        // 2. Compute (Independent Math)
        x = x * x;
        y = y * y;
        z = z * z;
        w = w * w;

        // 3. Store
        data[i] = x;
        data[i + stride] = y;
        data[i + stride * 2] = z;
        data[i + stride * 3] = w;
    }
}

int main() {
    int N = 1000000; // 1 Million floats
    size_t bytes = N * sizeof(float);

    float *d_data;
    cudaMalloc(&d_data, bytes);

    printf("--- ILP LATENCY HIDING BENCHMARK ---\n");
    printf("Constraint: Running with ONLY 1 WARP (32 Threads).\n");
    printf("The GPU cannot hide latency by switching warps.\n");
    printf("It MUST use Instruction Level Parallelism.\n\n");

    // Events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start); cudaEventCreate(&stop);
    float ms_serial, ms_ilp;

    // 1. Run Serial
    cudaEventRecord(start);
    serial_kernel<<<1, 32>>>(d_data, N); // 1 Block, 32 Threads
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms_serial, start, stop);

    printf("Serial Time: %6.2f ms\n", ms_serial);

    // 2. Run ILP
    cudaEventRecord(start);
    ilp_kernel<<<1, 32>>>(d_data, N);    // 1 Block, 32 Threads
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms_ilp, start, stop);

    printf("ILP Time:    %6.2f ms\n", ms_ilp);

    // 3. Results
    printf("\nSpeedup: %.2fx\n", ms_serial / ms_ilp);

    cudaFree(d_data);
    return 0;
}

Writing ilp_unrolling.cu


In [14]:
!nvcc -arch=sm_80 ilp_unrolling.cu -o ilp_test
!./ilp_test

--- ILP LATENCY HIDING BENCHMARK ---
Constraint: Running with ONLY 1 WARP (32 Threads).
The GPU cannot hide latency by switching warps.
It MUST use Instruction Level Parallelism.

Serial Time:  15.73 ms
ILP Time:      1.86 ms

Speedup: 8.47x


In [ ]:
%%writefile ilp_unrolling.cu
#include <stdio.h>
#include <cuda_runtime.h>


__global__ void serial_kernel(float *data, int N){
  int tid = threadIdx.x;
  int stride = blockDim.x;
  for (int i = tid, i < N; i+= stride){
    float x = data[i];
    x = x * x;
    data[i] = x;

  }
}